<a href="https://colab.research.google.com/github/rim-yu/procam-calibration/blob/master/Copy_of_cameraCalibrationCodeReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/"My Drive"/danbi-project/images

/content/drive/My Drive/danbi-project/images


In [0]:
pwd

'/content/drive/My Drive/danbi-project/images'

In [10]:
ls

 calibration_matrix.yaml  'image (16).JPG'  'image (22).JPG'  'image (5).JPG'
'image (10).JPG'          'image (17).JPG'  'image (23).JPG'  'image (6).JPG'
'image (11).JPG'          'image (18).JPG'  'image (24).JPG'  'image (7).JPG'
'image (12).JPG'          'image (19).JPG'  'image (25).JPG'  'image (8).JPG'
'image (13).JPG'          'image (1).JPG'   'image (2).JPG'   'image (9).JPG'
'image (14).JPG'          'image (20).JPG'  'image (3).JPG'
'image (15).JPG'          'image (21).JPG'  'image (4).JPG'


In [0]:
import numpy as np
import cv2
import glob
import yaml
import pathlib
from matplotlib import pyplot as plt 

# termination criteria 종료 기준. 
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((10*7, 3), np.float32)
objp[:,:2] = np.mgrid[0:10,0:7].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
# 패턴 검출을 통해 객체 지점(objpoints)와 이미지 지점(imgpoints)를 파악함. 
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('image *')
# images = glob.glob("graycode_*")
# path = 'results'
# pathlib.Path(path).mkdir(parents=True, exist_ok=True) 

In [0]:
from google.colab.patches import cv2_imshow

In [11]:
found = 0

for fname in images:  # Here, 10 can be changed to whatever number you like to choose
    img = cv2.imread(fname) # Capture frame-by-frame
    print(fname)
    #print(images[im_i])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners 
    # 체스보드의 패턴을 찾기 위해 cv2.findChessboardCorners() 함수를 사용한다. 
    # 이 함수는 코너 지점과 패턴이 발견되었는지 여부를 반환한다. 
    ret, corners = cv2.findChessboardCorners(gray, (10, 7), None)
    # If found, add object points, image points (after refining them)
    if ret == True: # 코너를 발견함. 
        objpoints.append(objp)   # Certainly, every loop objp is the same, in 3D.
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria) # 정확도를 높힘. 
        imgpoints.append(corners2)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (10,7), corners2, ret) # 결과를 그림. 
        found += 1
        # cv2_imshow(img) # -> colab에서 사용하기 힘든 함수라 함. 
        # cv2.waitKey(500)
        # plt.figure(figsize=(30, 30))
        # plt.imshow(img)
        # plt.axis("off")
        # plt.show()
        # if you want to save images with detected corners 
        # uncomment following 2 lines and lines 5, 18 and 19
        # image_name = path + '/calibresult' + str(found) + '.png'
        # cv2.imwrite(image_name, img)
    
print("Number of images used for calibration: ", found)

image (1).JPG
image (2).JPG
image (3).JPG
image (4).JPG
image (5).JPG
image (6).JPG
image (7).JPG
image (8).JPG
image (9).JPG
image (10).JPG
image (11).JPG
image (12).JPG
image (13).JPG
image (14).JPG
image (15).JPG
image (16).JPG
image (17).JPG
image (18).JPG
image (19).JPG
image (20).JPG
image (21).JPG
image (22).JPG
image (23).JPG
image (24).JPG
image (25).JPG
Number of images used for calibration:  25


In [0]:
len(imgpoints)

48

In [0]:
# When everything done, release the capture
# cap.release()
cv2.destroyAllWindows()

# calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# transform the matrix and distortion coefficients to writable lists
data = {'camera_matrix': np.asarray(mtx).tolist(),
        'dist_coeff': np.asarray(dist).tolist()}

# and save it to a file
with open("calibration_matrix.yaml", "w") as f:
    yaml.dump(data, f)

# done

In [13]:
print("RMS : ", ret)

RMS :  1.0503117463077551


In [0]:
np.asarray(mtx).tolist()

[[2111.8492406740324, 0.0, 1211.2940341075587],
 [0.0, 2056.731484626631, 883.6371360843018],
 [0.0, 0.0, 1.0]]

In [0]:
def printNumpyWithIndent(tar, indentchar):
    print(indentchar + str(tar).replace('\n', '\n' + indentchar))  

In [16]:
rvecs[2] # image(3)가 제일 평평함. 
cam_rotation_matrix = np.zeros(shape=(3, 3))
cv2.Rodrigues(rvecs[2], cam_rotation_matrix) 

print('  cam_rvecs_image(3) :')
printNumpyWithIndent(cam_rotation_matrix, '    ')

  cam_rvecs_image(3) :
    [[ 0.99996346 -0.0082431  -0.00226561]
     [ 0.00841565  0.90260298  0.43039172]
     [-0.00150281 -0.43039506  0.90263937]]


In [17]:
tvecs[2]

array([[-3.09891127],
       [-5.08157722],
       [30.73382527]])

**block size : default**

camera_matrix:

[[2111.8492406740324, 0.0, 1211.2940341075587]

[0.0, 2056.731484626631, 883.6371360843018]

[0.0, 0.0, 1.0]]

dist_coeff:

[[-0.4089409417093742, 2.4885790421166716, 0.043070452804298824, -0.0007310370007457026, -12.445596563012291]]

In [0]:
found = 0

for fname in images:  # Here, 10 can be changed to whatever number you like to choose
    img = cv2.imread(fname) # Capture frame-by-frame
    #print(images[im_i])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners 
    # 체스보드의 패턴을 찾기 위해 cv2.findChessboardCorners() 함수를 사용한다. 
    # 이 함수는 코너 지점과 패턴이 발견되었는지 여부를 반환한다. 
    ret, corners = cv2.findChessboardCorners(gray, (10, 7), None)
    # If found, add object points, image points (after refining them)
    if ret == True: # 코너를 발견함. 
        objpoints.append(objp)   # Certainly, every loop objp is the same, in 3D.
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria) # 정확도를 높힘. 
        imgpoints.append(corners2)
        # Draw and display the corners
        # img = cv2.drawChessboardCorners(img, (10,7), corners2, ret) # 결과를 그림. 
        found += 1
        # cv2.imshow('img', img) -> colab에서 사용하기 힘든 함수라 함. 
        # cv2.waitKey(500)
        # if you want to save images with detected corners 
        # uncomment following 2 lines and lines 5, 18 and 19
        # image_name = path + '/calibresult' + str(found) + '.png'
        # cv2.imwrite(image_name, img)

objpoints = [ num * 68 for num in objpoints ] # block size : 68 mm 
print("Number of images used for calibration: ", found)

Number of images used for calibration:  25


In [0]:
# When everything done, release the capture
# cap.release()
cv2.destroyAllWindows()

# calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# transform the matrix and distortion coefficients to writable lists
data = {'camera_matrix': np.asarray(mtx).tolist(),
        'dist_coeff': np.asarray(dist).tolist()}

# and save it to a file
with open("calibration_matrix.yaml", "w") as f:
    yaml.dump(data, f)

# done

In [0]:
print("RMS : ", ret)

RMS :  1.0503117463082716


In [0]:
np.asarray(mtx).tolist()

[[2111.8492406992, 0.0, 1211.294034109914],
 [0.0, 2056.7314846421655, 883.6371360622597],
 [0.0, 0.0, 1.0]]

In [0]:
np.asarray(rvecs).tolist()

[[[-0.32334001866066586], [0.36675296498226206], [1.7298601183985116]],
 [[-0.3972101890334299], [0.21066987191444192], [1.02874361159627]],
 [[-0.44493454055528314], [-0.0003942846021837501], [0.008610791485448292]],
 [[-0.4281532021664576], [-0.10976301009415405], [-0.512539363242263]],
 [[-0.26615099128102876], [-0.06255849109847178], [-0.3539125419373866]],
 [[-0.4225217607215091], [-0.1359643280888124], [-0.1345923695339664]],
 [[-0.3453879934523246], [0.26796333116458815], [1.322647275867424]],
 [[-0.25421665513827474], [0.39001566916795993], [1.8980051199331505]],
 [[-0.1406108456613008], [0.5751913848210738], [2.6873145823554414]],
 [[0.07493650258230795], [0.5615182784127009], [2.7654381553591154]],
 [[0.15127095136382931], [0.598423415966639], [3.032361358082064]],
 [[-0.2823514431289204], [-0.4641143162280463], [-2.651031625713511]],
 [[-0.40532998566635087], [-0.2562293094465907], [-2.0478578324785657]],
 [[-0.43700591391128746], [-0.18369780522277557], [-1.5653316508839328

In [0]:
np.asarray(tvecs).tolist()

[[[297.82479274258566], [-461.4911873082153], [2150.7520697482755]],
 [[87.02066033516644], [-580.7505123198864], [2198.085011989331]],
 [[-210.72596641663216], [-345.54725065687114], [2089.9001183155956]],
 [[-252.6745239219927], [-267.26562180753285], [2055.7873794044863]],
 [[-168.57709028852724], [-349.46984051172336], [2022.9352910148275]],
 [[-169.1056642796682], [-387.21525714431795], [2016.0812620180443]],
 [[265.63321786368954], [-508.5078046626103], [2145.8192549379296]],
 [[441.63313918094826], [-442.4038030267317], [2113.2092931160055]],
 [[561.5692420180504], [-268.29208575875936], [2045.6383858047711]],
 [[441.0983164248343], [-62.06887830529097], [1858.1372322168925]],
 [[407.2451402873558], [-0.6667895551015841], [1843.3643042122333]],
 [[344.7453195893706], [79.68123549119304], [1820.7113062861235]],
 [[158.6232135147529], [163.00613313542274], [1827.4320625315504]],
 [[-7.931011054768868], [78.52449052569796], [1875.2459109470221]],
 [[-208.88773798880703], [90.395313

**block size : 6.8 mm**

camera_matrix:

[[2111.8492406992, 0.0, 1211.294034109914]

[0.0, 2056.7314846421655, 883.6371360622597]

[0.0, 0.0, 1.0]]

dist_coeff:

[[-0.40894094173114914, 2.4885790426860726, 0.04307045280817715, -0.0007310370007799288,  -12.445596567134269]]